Импорты

In [ ]:
!pip install langdetect

In [1]:
import pandas as pd
from langdetect import detect

Соединеним файлы в  единый датасет. Для понимания на каком языке написан датасет, используем библиотеку langdetect

In [2]:
path = 'data/'

langs = []
langCount = {}
train_table = {'article': [], 'abstract': []}
ind = {}
for j in range(5):
    table = pd.read_parquet(path + f'train-0000{j}-of-00005.parquet')
    for i in table.values:
        if i[0] != '':
            train_table['article'].append(i[0])
            train_table['abstract'].append(i[1])

            lang = detect(i[1])
            if lang not in langs:
                langs.append(lang)
                langCount[lang]=1
                ind[lang] = i[1]
            else:
                langCount[lang]+=1
    print('iter:', j+1)


train = pd.DataFrame(train_table)
train


KeyboardInterrupt



Теперь можно узнать на каких языках написаны статьи, а также сколько статей написаны на определённом языке

In [30]:
langs, langCount

(['en', 'fr', 'it'], {'en': 117106, 'fr': 1, 'it': 1})

Почти все статьи написаны на английском. Выкинем лишние, чтобы улучшить качество обучения модели.

In [34]:
for i in langs:
    if i == 'en': continue
    train = train[train['abstract']!= ind[i]]
    
train.shape

(117106, 2)

In [71]:
#проверка на наличие заглавных букв
train['article'].apply(lambda x: x.istitle()).sum()

0

В ходе анализа данных мы поняли, что в датасете находятся статьи, связанные с медицинскими иследованиями, биологией, химией.
Главная особенность научных статей -  большое количество цифр, сокращений, вставных конструкций. Так, например в строке 65581 есть проценты, нумерование пунктов. В нулевой строке же в виде вставных конструкций фигурирует вероятность, которая обозначается буквой p. То есть модель должна научиться понимать сокращения и формулы. Проблемой так же может быть возникновение узконаправленных и редких терминов. В датасете так же было большое количество строк, на вход которых подавалась пустая строка, а на выходе был сокращенный текст. Данную проблему мы решили на момент загрузки датасета

В датасете в столбцах article и abstract нет заглавных букв, а так же все знаки препинания отделены пробелами, то есть датасет уже токенизирован. Это значит, что нам не нужно обрабатывать данные. При внедрении модели в продукт надо будет подавать ей токенизированный текст.


#### Выбор метрики
1. Метрика ROUGE
Метрика ROUGE состоит из recall, precision и f1-score для задач суммаризации, которые задаются следующим образом:

$$ recall = \frac{Количество\;слов,\;совпавших\;с\;abstract}{количество\;слов\;в\;abstract}$$

$$ precision = \frac{Количество\;слов\;совпавших\;с\;abstract}{количество\;сгенерированных\;слов}$$

$$ F1_{score} = 2*\frac{precision*recall}{precision+recall}$$

Плюсы данной метрики в легкой интерпретируемости и понятности.

2. Метрика сходства

Метрика сходства считает косинус угла между сгенерированным и эталонным (abstract) текстом, представляя данные предложения как вектора. Пусть A - вектор abstract, B - вектор сгенерированного текста

$$Similarity(A,B) = cos\theta = \frac{A*B}{|A|*|B|}$$

Данная метрика может показывать эффективность модели, однако интерпретировать её и понимать где недочеты (в отличие от тех же recall и precision в ROGUE) нельзя.

3. Метрика BLEU
Метрика BLEU состоит из precision и штрафа за краткость. Precision в случае BLEU рассчитывается следующим образом:

$$precision = \frac{Количество\;слов\;и\;словосочетаний,\;совпаших\;с\;abstract}{Количество\;слов\;и\;словосочетаний,\;совпаших\;в\;abstract}$$

Под словосочетанием в данном случае подразумевается пара рядом стоящих слов. Обозначим длину сгенерированного текста за c,а эталонного за r. Штраф  высчитывается следующим образом:
$$
BP = 
    \begin{cases}
    1, \;\;если \;c>r \\
    e^{(1-r/c)}, \;\;если \;c\leq r
    \end{cases}
$$

$$BLEU = BP * precision$$

Данная метрика, в отличие от остальных, подсчитывает ещё и словосочетания.


В ходе совместных раздумий мы решили использовать метрику ROUGE, а также при необходимости его модернизации, считающие словосочетания, так как мы считаем, что ROUGE самая понятная и легкоинтерпретируемая метрика. Precision и recall довольно точно могут показать недостатки модели,  которые могут появиться при обучении. Для подсчета метрики мы будем использовать библиотеку evaluate.